In [1]:
from __future__ import division
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl 
import numpy as np
from nltk import word_tokenize
from nltk import pos_tag
import csv,json

In [2]:
df = pd.read_csv("prod_raw.csv")

In [356]:
# remove the unwanted columns
df = df.drop(["RecordedDate", "ResponseId", "RecipientLastName","Finished","Duration (in seconds)","Progress","StartDate","EndDate","Status","IPAddress","RecipientFirstName","RecipientEmail","ExternalReference","LocationLatitude","LocationLongitude","DistributionChannel","UserLanguage","Q21"], axis=1)

In [357]:
# to rename the columns
# first take the names from the desired columns
names = df.iloc[0]
# rewrite the column names with those
df.columns = names

In [358]:
# drop unwanted rows
df = df.drop([0,1,2],axis=0)

# reset index and drop old index column
df = df.reset_index(drop=True)

In [359]:
df = df.rename(columns={'Please fill out the following fields. - Sona ID':'subject', 'Please fill out the following fields. - Native Language':'Nat_lang'})

In [360]:
# create a column of numbers to asign subject numbers to NaN values
nums = pd.Series(range(1,22))
# replace the NaN values with the series of numbers
df.subject = df.subject.fillna(value = nums)

In [361]:
# convert wide to long
df = pd.melt(df, id_vars=['subject','Nat_lang'], var_name='factors',value_name='response')

In [362]:
df["response"] = df.response.str.capitalize()

In [363]:
df

,subject,Nat_lang,factors,response
0,1,NaN,practice_na_lamp_na,What lamp will provide enough light for reading?
1,2,NaN,practice_na_lamp_na,Where can i find a lamp with good lighting?
2,3,NaN,practice_na_lamp_na,How bright is this lamp?
3,4,NaN,practice_na_lamp_na,Do you have a lamp that will provide enough li...
4,5,NaN,practice_na_lamp_na,What is your brightest lamp?
...,...,...,...,...
1945,16921,English,filler-na-potluck-na,What is a recipe for olive tapenade?
1946,16750,English,filler-na-potluck-na,How do you make olive tapenade?
1947,17659,English,filler-na-potluck-na,How do you make a olive tapenade?
1948,NaN,NaN,filler-na-potluck-na,NaN


In [364]:
# df.to_csv("wrangled.csv")

In [365]:
df['factors'] = df['factors'].replace('_','-', regex=True)

In [378]:
# df

In [367]:
# first split the facts column into the component factors
new = df['factors'].str.split('-', n=4, expand = True)

# then put them back into the original df with proper factor names
df["trial"] = new[0]
df["stakes"] = new[1]
df["story"] = new[2]
df["wh"] = new[3]

In [368]:
df = df.dropna(subset=['response'])
df = df.fillna({'Nat_lang':'na'})

In [377]:
df.subject.unique()

array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0,
       13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, '17617',
       '17026', '16918', '16570', '17926', '17899', '17245', 'jmc835',
       '18247', '17197', '18148', '16921 ', '16750', '17659', '17671'],
      dtype=object)

In [379]:
test = df.loc[df["stakes"].isin(['high','low'])]

In [380]:
test.head()

,subject,Nat_lang,factors,response,trial,stakes,story,wh
117,1,na,test-high-oyster-where,Where are the most contaminated oysters found?,test,high,oyster,where
118,2,na,test-high-oyster-where,Where are the oysters concentrated the most?,test,high,oyster,where
119,3,na,test-high-oyster-where,Where are the contaminated oysters?,test,high,oyster,where
120,4,na,test-high-oyster-where,Which restaurants sell oysters?,test,high,oyster,where
121,5,na,test-high-oyster-where,Where are the contaminated oysters?,test,high,oyster,where


In [383]:
high = df.loc[df["stakes"].isin(["high"])]
low = df.loc[df["stakes"].isin(["low"])]
oyster = df.loc[df["story"].isin(["oyster"])]

In [384]:
len(test)

576

In [385]:
avg = test.groupby(['stakes','story','response'])['response'].count()
avg.to_csv("grouped.csv", header=True)

In [386]:
can = test.loc[test["response"].str.contains("can")]
which = test.loc[test["response"].str.contains("Which")]
how = test.loc[test["response"].str.contains("How")]
who = test.loc[test.response.str.contains("Who")]
where = test.loc[test.response.str.contains("Where")]
what = test.loc[test.response.str.contains("What")]
polar = test.loc[test.response.str.contains("Are|Can|Do|Have|Is|Has")]
siri = test.loc[test.response.str.contains("Siri")]

In [388]:
polargroups = polar.groupby(['stakes'])['stakes'].count().apply(lambda x: 100 * x/len(test))
polargroups

stakes
high     7.465278
low     11.805556
Name: stakes, dtype: float64

In [389]:
whichgroups = which.groupby(['stakes'])['stakes'].count()
whichgroups.apply(lambda x: 100* x/len(which))

stakes
high    77.419355
low     22.580645
Name: stakes, dtype: float64

In [390]:
all = test.loc[test["response"].str.contains("all")]
some = test.loc[test["response"].str.contains("some")]
close = test.loc[test["response"].str.contains("close")]
local = test.loc[test["response"].str.contains("local")]

In [393]:
print(f"Total in Test: {len(test)}")
print(f"Occurrences of 'can' in test: {len(can)} \n\t Can: {len(can)/len(test)*100:.2f}%")
print(f"Occurrences of 'all': {len(all)} \n\t All: {len(all)/len(test)*100:.2f}%")
print(f"Occurrences of 'some': {len(some)} \n\t Some: {len(some)/len(test)*100:.2f}%")
print(f"Occurrences of 'close': {len(close)} \n\t Close: {len(close)/len(test)*100:.2f}%")
print(f"Occurrences of 'local': {len(local)} \n\t Local: {len(local)/len(test)*100:.2f}%")
print(f"Occurrences of 'which': {len(which)} \n\t Which: {len(which)/len(test)*100:.2f}%")
print(f"Occurrences of 'where': {len(where)} \n\t Where: {len(where)/len(test)*100:.2f}%")
print(f"Occurrences of 'who': {len(who)} \n\t Who: {len(who)/len(test)*100:.2f}%")
print(f"Occurrences of 'how': {len(how)} \n\t How: {len(how)/len(test)*100:.2f}%")
print(f"Occurrences of 'what': {len(what)} \n\t What: {len(what)/len(test)*100:.2f}%")
print(f"Occurrences of polar-q: {len(polar)} \n\t Polar: {len(polar)/len(test)*100:.2f}%")

 
print(f"Total with a wh-word: {(len(how)+len(which)+len(where)+len(who)+len(what))/len(test)*100:.2f}%")

Total in Test: 576
Occurrences of 'can' in test: 27 
	 Can: 4.69%
Occurrences of 'all': 20 
	 All: 3.47%
Occurrences of 'some': 15 
	 Some: 2.60%
Occurrences of 'close': 20 
	 Close: 3.47%
Occurrences of 'local': 10 
	 Local: 1.74%
Occurrences of 'which': 62 
	 Which: 10.76%
Occurrences of 'where': 226 
	 Where: 39.24%
Occurrences of 'who': 15 
	 Who: 2.60%
Occurrences of 'how': 39 
	 How: 6.77%
Occurrences of 'what': 113 
	 What: 19.62%
Occurrences of polar-q: 111 
	 Polar: 19.27%
Total with a wh-word: 78.99%


In [394]:
# first rename the 'response' column so that the parser can run
df = df.rename(columns={'response':'sentence'})
# save as csv
df.to_csv("wrangled.csv")

In [261]:
# # write some functions to label these questions
# # for questType

# def label_quest (row) : 
#     if row["response"].str.contains('Which'):
#         return "which"
#     if row["response"].str.contains('which'):
#         return "which"
#     if row["response"].str.contains("Can"):
#         return "polar"
#     if row["response"].str.contains("Do"):
#         return "polar"
#     if row["response"].str.contains("Is"):
#         return "polar"
#     if row["response"].str.contains("Are"):
#         return "polar"
#     if row["response"].str.contains("Have"):
#         return "polar"
#     if row["response"].str.contains("Do"):
#         return "polar"
#     if row["response"].str.contains("can"):
#         return "modal"
#     if row["response"].str.contains("Who"):
#         return "who"
#     if row["response"].str.contains("who"):
#         return "who"
#     if row["response"].str.contains("How"):
#         return "how"
#     if row["response"].str.contains("how"):
#         return "how"
#     if row["response"].str.contains("Where"):
#         return "where"
#     if row["response"].str.contains("where"):
#         return "where"
#     if row["response"].str.contains("What"):
#         return "what"
#     if row["response"].str.contains("what"):
#         return "what"
# #     if row["response"].str.contains("all"):
# #         return "universal"
# #     if row["response"].str.contains("some"):
# #         return "existential"
# #     if row["response"].str.contains("the"):
# #         return "definite"
# #     if row["response"].str.contains("an"):
# #         return "indefinite"
# #     if row["response"].str.contains("local"):
# #         return "local"
# #     if row["response"].str.contains("close"):
# #         return "close"
#     return "other"
    

In [386]:
# df["qType"] = df.apply(lambda x: label_quest(x))

In [353]:
# Convert CSV to JSON for running the parser
csvFilePath = 'wrangled.csv'
jsonFilePath = 'production.json'

j = pd.read_csv(csvFilePath)

with open(jsonFilePath, 'w') as jas:
    jas.write(j.to_json(orient='records'))